In [2]:
import time
import random
from ortools.linear_solver import pywraplp  # Google OR Tools - Linear Solver

solver = pywraplp.Solver.CreateSolver('SCIP')  # Solver object to use "SCIP" algorithm of the linear solver 

# Problem Data
* Lim wants to pack items as many as possible into his backpack. At the same time, xe wants to maximise the total value of the items packed into it.
* The backpack can carry items within its area limit.
    - max_area: area of the backpack
* There are N distinct items, each of which has its own weight (or value) and dimension (width & depth).
    - value[i]: value of the i-th item
    - width[i]: width of the i-th item
    - depth[i]: depth of the i-th item
    - area[i] = width[i] * depth[i]: area of the i-th item
* For now, let's forget about the tetris-like problem here as it's over-complicated at this learning stage.

In [3]:
# Items
number_of_items = 50
item = {}
for i in range(number_of_items):  # Each item has its value & area (= width * depth)
    item[i] = {'value': random.randint(1, 5), 'width': random.randint(10, 30), 'depth': random.randint(10, 30)}

# Backpack
max_area = 5000  # Maximum area of the backpack

# Variables
* Binary decision variable: x[i] = 1 if the i-th item is selected to be packed or 0 otherwise
* We have N items, so we have N binary decision variables, x[1], x[2], ..., x[N].

In [4]:
x = {}  # Binary decision variable, x[i] = 1 if i-th item is selected to be packed or x[i] = 0 otherwise
for i in range(number_of_items):
    x[i] = solver.BoolVar(f'x_{i}')

# Constraints
* The backpack's area is limited. So, we need to make a decision which items to pack into the backpack.
* The complication here is that items have different values & dimensions.
* Anyway, we can model the constraint like this
    - area[1] * x[1] + area[2] * x[2] + ... + area[N] * x[N] <= max_area
    - sum{ area[i] * x[i] | i = 1, 2, ..., N } <= max_area, equivalently

In [5]:
# Constraint: total area of the items to be packed must be no greater than the backpack's area
solver.Add(
    sum(item[i]['width'] * item[i]['depth'] * x[i] for i in range(number_of_items)) <= max_area
)

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7f6b381d6b10> >

# Objective Function
* Lim wants to maximize the total value of the items packed into the backpack.
* So, the objective function needs to take the total value of the selected items into account in its calculation.
    - Maximize ( value[1] * x[1] + value[2] * x[2] + ... + value[N] * x[N] )
    - equivalently, Maximize sum{ value[i] * x[i] | i = 1, 2, ..., N }

In [6]:
# Maximise the total weight of the items to be packed
solver.Maximize(
    sum(item[i]['value'] * x[i] for i in range(number_of_items))
)

# Solve
### Linear Programming Model
- Maximize
    - sum{ value[i] x[i] | i = 1, 2, ..., N }
- Subject to
    - sum{ width[i] * depth[i] * x[i] | i = 1, 2, ..., N } ≤ max_area
    - x[i] ∈ {0, 1} for i = 1, 2, ..., N

In [8]:
# Solve the linear program
status = solver.Solve()

# Check the results
print('status =', status)  # 0=optimal, 1=feasible, 2=infeasible, 3=unbounded, 4=abnormal, 6=not solved
print('objective value =', solver.Objective().Value())
number_of_items_packed = 0
total_area_used = 0
for i in range(number_of_items):
    if x[i].solution_value() == 1:
        total_area_used += item[i]["width"] * item[i]["depth"]
        number_of_items_packed += 1
print('total items packed =', number_of_items_packed)
print('total area used =', total_area_used)

status = 0
objective value = 71.0
total items packed = 17
total area used = 4865
